# make dataset

#### I got the imagelevel dataframe by [step2 make_dataframe](https://www.kaggle.com/kunihikofurugori/siim-step1-get-imginfo).

#### Next step, I will make the dataset about image level.

## my public notebook 

### [step1 get_imageinformation](https://www.kaggle.com/kunihikofurugori/siim-step1-get-imginfo).

### [step2 make_dataframe](https://www.kaggle.com/kunihikofurugori/siim-step1-get-imginfo).

### [step3-1 renew-imglev_ds](https://www.kaggle.com/kunihikofurugori/siim-step3-1-renew-imglev-ds)

### [step3-2 renew-studylev_ds](https://www.kaggle.com/kunihikofurugori/siim-step3-1-renew-studylev-ds)

In [ ]:
!pip install python-gdcm
#!pip install pylibjpeg-libjpeg
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2,random
import pydicom
import gdcm
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import glob
import os
from PIL import Image, ImageStat
import albumentations as A

In [ ]:
os.mkdir("image640")
os.mkdir("image768")

In [ ]:
df = pd.read_csv("../input/step2-make-dataframe/train_image.csv")
print(len(df))
df = df[~df.duplicated(keep='last',subset=['mean', 'var'])]
print(len(df))
df = df.sort_values("norm_var",ascending=False)
df['fold'] = 0
df.fold = df.fold.apply(lambda x: random.randint(0,3))
df

In [ ]:
def get_transform(size):
    transform = A.Compose([
        #A.Rotate(p=1.0,limit=5),
        A.RandomSizedCrop(
                         min_max_height=[int(0.8*size), int(0.95*size)],
                         height=size,
                         width=size, 
                         p=1.0)
    ],p=1.0,
    bbox_params=A.BboxParams(
                    format='albumentations',
                    min_area=0, 
                    min_visibility=0,
                    label_fields=['label'])
    )
    return transform


def exclude_blackflame(image, boxes, labels, size, trytimes=10):
    best_var = image.std()
    transform = get_transform(size)
    for _ in range(trytimes):
        sample = transform(image=image, bboxes=boxes, label=labels)
        if sample['image'].std() < best_var:
            best_var = sample['image'].std()
            images = sample['image']
            bboxes = sample['bboxes']
    if best_var >= image.std():
        images = image
        bboxes = boxes
    return images, bboxes

In [ ]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

def resize(array, size1,size2, keep_ratio=True, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size1, size2), resample)
        im = expand2square(im, background_color=0)
    else:
        im = im.resize((size1, size2), resample)
    
    return im

In [ ]:
dfs = []
dfs.append(df[df.fold ==0])
dfs.append(df[df.fold ==1])
dfs.append(df[df.fold ==2])
dfs.append(df[df.fold ==3])
print(len(dfs[0]),len(dfs[1]),len(dfs[2]),len(dfs[3]))
print(len(df))

In [ ]:
def run(fold):
    for idx,(index,row) in enumerate(dfs[fold].iterrows()):
        image = read_xray(row.path)
        boxes = np.zeros((row.boxnum,4))
        labels =np.ones((boxes.shape[0],), dtype=np.int64)
        for i in range(row.boxnum):
            box = row[['fracxmin'+str(i+1), 'fracymin'+str(i+1), 'fracxmax'+str(i+1), 'fracymax'+str(i+1)]].values
            boxes[i] = box
        image = cv2.resize(image,(2048,2048),interpolation = cv2.INTER_AREA)
        save_img = cv2.resize(image,(640,640),interpolation = cv2.INTER_AREA)
        cv2.imwrite("image640" + "/" + row.id + ".png",save_img)
        save_img = cv2.resize(image,(768,768),interpolation = cv2.INTER_AREA)
        cv2.imwrite("image768" + "/" + row.id + ".png",save_img)
                
        #plt.imshow(images)
        #plt.show()

        if idx%500==0:
            print(idx)

In [ ]:
#run(1)

In [ ]:
from joblib import Parallel, delayed
Parallel(n_jobs=4, backend="threading")(delayed(run)(i) for i in range(4))

In [ ]:
!zip -qr image640.zip image640
!rm -r image640
!zip -qr image768.zip image768
!rm -r image768

In [ ]:
df.to_csv("train.csv")